In [1]:
from dotenv import dotenv_values
from selenium import webdriver
from selenium.webdriver.firefox.webdriver import WebDriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import *
from selenium.webdriver import Keys, ActionChains
from shutil import move
import os
from pathlib import Path
import datetime
import time

In [2]:

class openBrowserCrefisa():
    
    # def __init__(self, user_credencials: dict = dotenv_values("./data/.env")) -> WebDriver:
    def __init__(self, user_credencials: dict = dotenv_values("./data/.env")) -> WebDriver:
        self.user_credencials_login = user_credencials['LOGIN_USER']
        self.user_credencials_password = user_credencials['LOGIN_PASSWORD']
        self.mkDirDownload = self.__mkDirDownload()
        
        
        
    
    def __mkDirDownload(self) -> None:
        os.makedirs("download", exist_ok=True)
        for file in os.listdir('download'):
            os.remove(f"download\\{file}")
            
    def __initialize_driver(self):
        
        options = Options()
        options.binary_location = r"C:\Program Files\Mozilla Firefox\firefox.exe"
        # options.add_argument('-headless')
        options.set_preference("browser.download.folderList", 2)
        options.set_preference("browser.download.manager.showWhenStarting", False)
        options.set_preference("browser.download.dir", fr"{str(Path().absolute())}\\download")
        options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/x-gzip")
        driver = webdriver.Firefox(options=options)
        driver.maximize_window()
        return driver
    
    def loguinSendKeys(self):
        driver = self.__initialize_driver()
        driver.get('https://app1.gerencialcredito.com.br/CREFISA/')
        WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#txtUsuario'))).send_keys(self.user_credencials_login)
        time.sleep(1)
        WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#txtSenha'))).send_keys(self.user_credencials_password)
        time.sleep(1)
        WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#btnLogin'))).click()
        return driver
        

    


In [3]:
class get_tables():


    def __init__(self, driver: WebDriver, date: datetime.date, user_credencials= dotenv_values("./data/.env")):
        self.driver = driver
        self.user_autentiction = user_credencials['USER_AUTHENTICATION']
        self.date = date
        


    def __move_file(self, date: datetime.date):
        path_download = f"./download/"
        path_to_move = f'./relatorios/{date.year}/{date.month}'
        os.makedirs(path_to_move, exist_ok = True)
        if os.path.exists(path_download):
            while os.listdir(path_download) == 0:
                continue
            time.sleep(2)
            move(path_download + os.listdir(path_download)[0], path_to_move + f"/relatorio_{date.day}_{date.month}_{date.year}.xls")


    def download(self):
        date_work = self.date
        while date_work  < datetime.date.today():


            WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#menu4'))).click()
            
            
            WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#listaMenu4 > li:nth-child(2) > a:nth-child(1)'))).click()

            try:
                WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#txtAutenticacaoSenhaFinanceira'))).send_keys(self.user_autentiction)
                WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'button.btn:nth-child(2)'))).click()
                WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.swal2-confirm'))).click()
            except TimeoutException:
                pass            
            WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#txtDataInicial'))).clear()
            WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#txtDataInicial'))).click()
            WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#txtDataInicial'))).send_keys(date_work.strftime("%d/%m/%Y"))
            WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#txtDataFinal'))).clear()
            WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#txtDataFinal'))).click()
            WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#txtDataFinal'))).send_keys(date_work.strftime("%d/%m/%Y"))
            WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#ddlTipoResultado'))).click()

            WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#ddlTipoDeData > option:nth-child(2)'))).click()
            WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#ddlTipoDeData > option:nth-child(2)'))).click()

            WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#ddlStatusPagCliente > option:nth-child(3)'))).click()
            WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#ddlStatusPagCliente > option:nth-child(3)'))).click()
            WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#ddlTipoResultado > option:nth-child(1)'))).click()
            WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#ddlTipoResultado > option:nth-child(1)'))).click()
            WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#btn'))).click()
            
            try:
                allert = self.driver.switch_to.alert
                self.driver_alert = allert.text

                if self.driver_alert == 'NENHUM RESULTADO FOI ENCONTRADO.':
                    print("Sem dados para: ", date_work)
                    allert.dismiss()
                    date_work = date_work + datetime.timedelta(days = 1)
                    continue

                else:
                    allert = self.driver.switch_to.alert
                    allert.accept()
                    p = self.driver.current_window_handle
                    chwd = self.driver.window_handles
                    for w in chwd:

                        if(w!=p):
                            self.driver.switch_to.window(w)
                            break

            except NoAlertPresentException:
                pass

            self.__move_file(date =date_work)
            date_work
            date_work = date_work + datetime.timedelta(days = 1)

        self.driver.quit()
        


In [4]:

teste  = get_tables(driver = openBrowserCrefisa().loguinSendKeys(), date = datetime.date(2024, 3, 21)).download()

Sem dados para:  2024-03-23
Sem dados para:  2024-03-24


In [10]:
dados = dotenv_values("./data/.env")

In [11]:
dados

OrderedDict([('LOGIN_USER', '11851164600'),
             ('LOGIN_PASSWORD', 'vA4g4Gqw1a'),
             ('USER_AUTHENTICATION', '1010')])

In [20]:
WebDriverWait(teste, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.swal2-confirm'))).click()
WebDriverWait(teste, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#txtDataInicial'))).click()
WebDriverWait(teste, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#txtDataInicial'))).send_keys("01/03/2024")
WebDriverWait(teste, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#txtDataFinal'))).click()
WebDriverWait(teste, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#txtDataFinal'))).send_keys("02/03/2024")
WebDriverWait(teste, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#ddlTipoResultado'))).click()
WebDriverWait(teste, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#ddlTipoResultado > option:nth-child(1)'))).click()
WebDriverWait(teste, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#ddlTipoResultado > option:nth-child(1)'))).click()
WebDriverWait(teste, 20).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#btn'))).click()



In [3]:
import pandas as pd

NameError: name 'teste' is not defined

In [8]:
teste_df = pd.DataFrame(teste[0])

Index(['FILIAL', 'GRUPO_VENDEDOR', 'COD_VENDEDOR', 'VENDEDOR', 'CONTRATO_ID',
       'NUMERO_ADE', 'NUMERO_CONTRATO', 'PRAZO', 'VLR_PARC', 'VALOR_BRUTO',
       'VALOR_LIQUIDO', 'VALOR_BASE', 'PERC_COMISSAO_REPASSE',
       'VLR_COMISSAO_REPASSE', 'CLIENTE', 'CPF', 'SIT_BANCO',
       'SIT_PAGAMENTO_CLIENTE', 'DATA_DIGIT_BANCO', 'DATA_PAGAMENTO_CLIENTE',
       'BANCO', 'CONVENIO', 'TABELA', 'USUARIO_DIGIT_BANCO',
       'USUARIO_DIGIT_BANCO_SUBESTABELECIDO', 'SUB_USUARIO',
       'LOGIN_SUB_USUARIO', 'SITUACAO_PENDENCIA', 'TIPO CONTRATO',
       'CODIGO_PRODUTO', 'CODIGO_CONVENIO', 'FISICO_EMPRESA',
       'DATA_FISICO_EMPRESA', 'USUÁRIO_FISICO_EMPRESA',
       'SIT_PAGAMENTO_COMISSAO', 'DATA_PAGAMENTO_COMISSAO', 'SUB_STATUS',
       'DATA_SUB_STATUS', 'PERC_BONUS_REPASSE', 'VLR_BONUS_REPASSE'],
      dtype='object')